In [23]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [24]:
# Create the model
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash-exp",
    generation_config=generation_config,
)

chat_session = model.start_chat(history=[])

In [25]:
from tqdm.notebook import tqdm
import time
import logging

logger = logging.getLogger(__name__)

def localtime_to_str():
    return time.strftime('%Y-%m-%d_%H-%M-%S', time.localtime())

logging.basicConfig(
    filename=f"data/output/gemini/log/gemini_{localtime_to_str()}.log", 
    encoding="utf-8",
    level=logging.DEBUG
    )

In [26]:
prompt_condition_of_the_incident = """
### 指示 ###
あなたは製品の事故原因を分析しています。
事故の通知内容から、被害の様子を単純に説明してください。
回答形式はJSON形式です。

### 回答形式 ###
{
	"被害":"回答"
}

### 注意事項 ###
- 指示された回答のみ出力してください。
- 被害の様子を最もよく説明する単語のみで回答してください。
- 被害は複数回答してもかまいません。その場合、配列で記述してください。
- 解説は不要です。

### 例 ###
事故の通知内容:
カセットこんろの五徳付プレートを洗っていたところ、指が切れ、第一関節裏側を2針縫った。

回答:
{
	"被害":"裂傷"
}

### 例の解説 ###
「指が切れ」という記述から、「裂傷」を負ったと判断できます。

### 事故の通知内容 ###
"""

In [27]:
prompt_cause_of_the_incident = """
### 指示 ###
あなたは製品の事故原因を分析しています。
事故の内容から、事故の原因を単純に説明してください。
回答形式はJSON形式です。

### 回答形式 ###
{
	"原因":"回答"
}

### 注意事項 ###
- 指示された回答のみ出力してください。
- 事故の原因を最もよく説明する単語のみで回答してください。
- 原因は複数回答してもかまいません。その場合、配列で記述してください。
- 解説は不要です。

### 例 ###
事故の内容:
五徳付プレートの外側の縁が処理されておらず、多少ざらついていることから、素手でプレートを洗っていた際に、外側の縁に指を強く擦りつけたため、負傷したものと推定される。なお、取扱説明書には「手、指の保護のため、必ずゴム手袋などを使用してください。」との注意が記載されている。

回答:
{
	"原因":["縁","接触"]
}

### 例の解説 ###
「縁に指を強く擦りつけた」という記述から、「縁」、「接触」が原因であると判断できます。

### 事故の内容 ###
"""

In [28]:
import pandas as pd
import json

In [29]:
global former_data
former_data = ""

In [30]:
def generate_json_response(
        prompt:str, 
        data:pd.DataFrame, 
        response_name:str, 
        chat_session:genai.ChatSession,
        request_interval:int=6
        ) -> None:
    global former_data
    for i in tqdm(range(0, len(data))):
        if data.values[i] == former_data:
            logger.debug(f"Skipped: {data.values[i]}")
            data.values[i] = data.values[i-1]
            continue
        input_prompt = prompt + data.values[i]
        former_data = data.values[i]
        try:
            response = chat_session.send_message(input_prompt)
            temp = response.text
            if "```json" in temp:
                temp = temp.replace("```json", "")
                temp = temp.replace("```", "")
            res = json.loads(temp)
            logger.debug(temp)
        except Exception as e:
            logger.error("Error", e, temp)
            continue
        output_text = ""
        if isinstance(res[response_name], list):
            for text in res[response_name]:
                output_text += text + " "
        else:
            output_text = res[response_name]
        logger.debug(f"Output: {output_text}")
        data.values[i] = output_text
        time.sleep(request_interval)

In [31]:
import settings as st

In [32]:
def processor(product:str)->None:
    data = pd.read_csv(f"data/output/gemini/prompt_data/{product}.csv")
    generate_json_response(
        prompt=prompt_condition_of_the_incident, 
        data=data["事故通知内容"], 
        response_name="被害", 
        chat_session=chat_session
        )
    generate_json_response(
        prompt=prompt_cause_of_the_incident, 
        data=data["事故原因"], 
        response_name="原因", 
        chat_session=chat_session
        )
    print(data.head())
    data.to_csv(f"data/output/gemini/responses/{product}_res.csv", index=False)

In [33]:
processor(st.products_250115[5])

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

     No     事故通知内容              事故原因
0  1043  巻き込まれ 負傷   紙詰まり 押し込み 引き込まれ 
1  1640         焼損     潤滑剤 可燃性ガス 引火 
2  2410         負傷    CD穴 挟み込み 引き抜き 
3  2487     出火 焼損     分解 エアースプレー 引火 
4  2693     出火 負傷   エアゾール缶 可燃性ガス 引火 
